In [3]:
import SisterMachineDataPipeline as ssc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import scipy.stats as stats
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
import glob

# For Mac
infiles_sisters = glob.glob(r'/Users/alestawsky/PycharmProjects/untitled/SISTERS/*.xls')
infiles_nonsisters = glob.glob(r'/Users/alestawsky/PycharmProjects/untitled/NONSISTERS/*.xls')
infiles_sisters_protein = glob.glob(r'/Users/alestawsky/PycharmProjects/untitled/ALLSIS/*.xls')
infiles_nonsisters_protein = glob.glob(r'/Users/alestawsky/PycharmProjects/untitled/AllNONSIS/*.xls')

# data_trap_median = ssc.SisterCellData(infiles_sister=infiles_sisters, infiles_nonsister=infiles_nonsisters, discretization_variable='length', infiles_sister_protein=infiles_sisters_protein,
#         infiles_nonsister_protein=infiles_nonsisters_protein, what_to_subtract='trap median', start_index=None, end_index=None, datasets=['Population', 'Sisters', 'Nonsisters', 'Control'])
# print('data_trap_median')

# data_trap_mean = ssc.SisterCellData(infiles_sister=infiles_sisters, infiles_nonsister=infiles_nonsisters, discretization_variable='length', infiles_sister_protein=infiles_sisters_protein,
#         infiles_nonsister_protein=infiles_nonsisters_protein, what_to_subtract='trap mean', start_index=None, end_index=None, datasets=['Population', 'Sisters', 'Nonsisters', 'Control'])
# print('data_trap_mean')
data_trap_mean = ssc.SisterCellData(infiles_sister=infiles_sisters, infiles_nonsister=infiles_nonsisters, discretization_variable='length', infiles_sister_protein=infiles_sisters_protein,
        infiles_nonsister_protein=infiles_nonsisters_protein, what_to_subtract='trap mean', start_index=None, end_index=None, datasets=['Sisters'])
print('data_trap_mean')

# data_global_median = ssc.SisterCellData(infiles_sister=infiles_sisters, infiles_nonsister=infiles_nonsisters, discretization_variable='length', infiles_sister_protein=infiles_sisters_protein,
#         infiles_nonsister_protein=infiles_nonsisters_protein, what_to_subtract='global median', start_index=None, end_index=None, datasets=['Population', 'Sisters', 'Nonsisters', 'Control'])
# print('data_global_median')
# 
# data_global_mean = ssc.SisterCellData(infiles_sister=infiles_sisters, infiles_nonsister=infiles_nonsisters, discretization_variable='length', infiles_sister_protein=infiles_sisters_protein,
#         infiles_nonsister_protein=infiles_nonsisters_protein, what_to_subtract='global mean', start_index=None, end_index=None, datasets=['Population', 'Sisters', 'Nonsisters', 'Control'])
# print('data_global_mean')



there's been a negative or zero growth_rate found!


length_final <= length_birth, and this so-called "generation" was taken out of its dataframe


Sister Generation data Finished


Got the pooled dataframe for Sisters


Got the log pooled dataframe for Sisters


got the intragenerational_dataframes for Sister


Got the _all_raw_protein_data_dict and A/B_raw_protein_data_dict attribute in Sister
data_trap_mean


In [5]:
""" labeled data entropy """

def get_entropies_from_labeled_data(A_cut, B_cut, bins_on_side, base_of_log):
    joint_centered = pd.DataFrame({'A': A_cut, 'B': B_cut})

    joint_prob_list = dict([('{}_{}'.format(label_A, label_B), len(joint_centered[(joint_centered['A'] == label_A) & (joint_centered['B'] == label_B)]) / len(joint_centered)) for label_B in
                            np.arange(2 * bins_on_side) for label_A in np.arange(2 * bins_on_side)])
    A_trace_marginal_probs = dict([('{}'.format(label_A), len(A_cut.iloc[np.where(A_cut == label_A)]) / len(A_cut)) for label_A in np.arange(2 * bins_on_side)])
    B_trace_marginal_probs = dict([('{}'.format(label_B), len(B_cut.iloc[np.where(B_cut == label_B)]) / len(B_cut)) for label_B in np.arange(2 * bins_on_side)])

    # conditioning the A trace based on the B trace
    A_conditioned_on_B_entropy = np.array([- joint_prob_list[key] * (np.log(joint_prob_list[key] / A_trace_marginal_probs[key.split('_')[0]]) / np.log(base_of_log)) for key in joint_prob_list.keys() if
                                           joint_prob_list[key] != 0 and A_trace_marginal_probs[key.split('_')[0]] != 0]).sum()
    B_conditioned_on_A_entropy = np.array([- joint_prob_list[key] * (np.log(joint_prob_list[key] / B_trace_marginal_probs[key.split('_')[1]]) / np.log(base_of_log)) for key in joint_prob_list.keys() if
                                           joint_prob_list[key] != 0 and B_trace_marginal_probs[key.split('_')[1]] != 0]).sum()

    # the mutual information between A and B trace for this variable thinking that marginal A and B came from each trace distribution
    mutual_info_trace = round(np.array(
        [joint_prob_list[key] * (np.log(joint_prob_list[key] / (A_trace_marginal_probs[key.split('_')[0]] * B_trace_marginal_probs[key.split('_')[1]]))/ np.log(base_of_log)) for key in joint_prob_list.keys() if
         joint_prob_list[key] != 0 and B_trace_marginal_probs[key.split('_')[1]] != 0 and A_trace_marginal_probs[key.split('_')[0]] != 0]).sum(), 14)

    # checking joint prob adds up to one
    if round(np.array([val for val in joint_prob_list.values()]).sum(), 4) != 1.0:
        print('joint prob does not add up to 1.0! it adds up to {}'.format(np.array([val for val in joint_prob_list.values()]).sum()))
        exit()

    # checking A marginal prob adds up to one
    if round(np.array([val for val in A_trace_marginal_probs.values()]).sum(), 4) != 1.0:
        print('A_trace_marginal_probs does not add up to 1.0! it adds up to {}'.format(np.array([val for val in A_trace_marginal_probs.values()]).sum()))
        exit()

    # checking B marginal prob adds up to one
    if round(np.array([val for val in B_trace_marginal_probs.values()]).sum(), 4) != 1.0:
        print('B_trace_marginal_probs does not add up to 1.0! it adds up to {}'.format(np.array([val for val in B_trace_marginal_probs.values()]).sum()))
        exit()

    # mutual information cannot be negative
    if mutual_info_trace < 0:
        print('mutual info is negative! something is wrong...')
        print(A_cut)
        print(B_cut)
        print(joint_prob_list)
        print(A_trace_marginal_probs)
        print(B_trace_marginal_probs)
        print(mutual_info_trace)
        for key in joint_prob_list.keys():
            print('key:', key)
            if joint_prob_list[key] != 0 and B_trace_marginal_probs[key.split('_')[1]] != 0 and A_trace_marginal_probs[key.split('_')[0]] != 0:
                print(joint_prob_list[key])
                print(A_trace_marginal_probs[key.split('_')[0]])
                print(B_trace_marginal_probs[key.split('_')[1]])
                print(joint_prob_list[key] * np.log(joint_prob_list[key] / (A_trace_marginal_probs[key.split('_')[0]] * B_trace_marginal_probs[key.split('_')[1]])))
        print('_________')
        exit()

    return mutual_info_trace, A_conditioned_on_B_entropy, B_conditioned_on_A_entropy, joint_prob_list

A_categories, B_categories = data_trap_mean.put_the_categories(A_df=data_trap_mean.sis_A_pooled, B_df=data_trap_mean.sis_B_pooled, log_A_df=data_trap_mean.sis_log_A_pooled, 
    log_B_df=data_trap_mean.sis_log_B_pooled, bins_on_side=1, variable_names=data_trap_mean._variable_names, log_vars=['generationtime', 'length_birth', 'length_final', 'growth_rate', 'fold_growth', 'added_length'])

print(A_categories)

for var1 in data_trap_mean._variable_names:
    for var2 in data_trap_mean._variable_names:
        MI_array = data_trap_mean.get_entropies_from_labeled_data(A_cut=A_categories[var1], B_cut=A_categories[var2], bins_on_side=1, base_of_log=2)
        print(MI_array)

     generationtime length_birth length_final growth_rate fold_growth  \
0                 0            1            1           0           0   
1                 0            1            1           1           0   
2                 0            1            1           1           1   
3                 1            1            1           1           1   
4                 0            1            1           1           0   
5                 0            1            1           0           0   
6                 0            1            0           0           0   
7                 1            0            0           0           1   
8                 1            0            0           1           1   
9                 1            0            1           1           1   
10                1            1            1           0           1   
11                0            1            1           0           0   
12                0            1            0      

NameError: name 'exit' is not defined